In [1]:
import os
import json
import random
import re
import time
from typing import Dict, List, Tuple, Optional
import logging
from dotenv import load_dotenv
from openai import OpenAI

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("chatbot_evaluation.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OPENAI_API_KEY environment variable not set")

client = OpenAI(api_key=api_key)

# Constants
VIETNAMESE_FIRST_NAMES = [
    "Anh", "Bình", "Cường", "Dung", "Hà", "Hải", "Hương", "Lan", "Linh", "Mai", 
    "Minh", "Nam", "Nga", "Phương", "Quân", "Thảo", "Thủy", "Trang", "Tuấn", "Việt"
]

VIETNAMESE_LAST_NAMES = [
    "Nguyễn", "Trần", "Lê", "Phạm", "Hoàng", "Huỳnh", "Phan", "Vũ", "Võ", "Đặng",
    "Bùi", "Đỗ", "Hồ", "Ngô", "Dương", "Lý", "Đào", "Đinh", "Trịnh", "Mai"
]

# User scenario templates
USER_SCENARIOS = [
    "Đang tìm một chiếc điện thoại mới với camera tốt",
    "Muốn mua điện thoại có pin trâu để chơi game",
    "Cần một chiếc điện thoại giá rẻ cho người lớn tuổi",
    "Muốn nâng cấp từ điện thoại cũ lên smartphone cao cấp",
    "Đang tìm hiểu về các dòng iPhone mới nhất",
    "Muốn mua điện thoại Samsung có màn hình lớn",
    "Cần tư vấn về điện thoại phù hợp với ngân sách 5 triệu đồng",
    "Đang so sánh giữa iPhone và Samsung",
    "Muốn mua trả góp một chiếc điện thoại cao cấp",
    "Cần một chiếc điện thoại có bộ nhớ lớn để lưu nhiều ảnh và video"
]

def generate_vietnamese_name() -> str:
    """Generate a random Vietnamese name."""
    first_name = random.choice(VIETNAMESE_FIRST_NAMES)
    last_name = random.choice(VIETNAMESE_LAST_NAMES)
    return f"{last_name} {first_name}"

def generate_vietnamese_phone() -> str:
    """Generate a valid Vietnamese phone number (10 digits)."""
    prefixes = ["032", "033", "034", "035", "036", "037", "038", "039",  # Viettel
                "070", "076", "077", "078", "079",  # Mobifone
                "081", "082", "083", "084", "085"]  # Vinaphone
    
    prefix = random.choice(prefixes)
    suffix = ''.join(random.choices('0123456789', k=10-len(prefix)))
    return prefix + suffix

def generate_email(name: str) -> str:
    """Generate an email based on name."""
    # Simplify Vietnamese characters
    simplified_name = name.lower()
    vietnamese_chars = {
        'à': 'a', 'á': 'a', 'ả': 'a', 'ã': 'a', 'ạ': 'a', 
        'ă': 'a', 'ằ': 'a', 'ắ': 'a', 'ẳ': 'a', 'ẵ': 'a', 'ặ': 'a',
        'â': 'a', 'ầ': 'a', 'ấ': 'a', 'ẩ': 'a', 'ẫ': 'a', 'ậ': 'a',
        'đ': 'd',
        'è': 'e', 'é': 'e', 'ẻ': 'e', 'ẽ': 'e', 'ẹ': 'e',
        'ê': 'e', 'ề': 'e', 'ế': 'e', 'ể': 'e', 'ễ': 'e', 'ệ': 'e',
        'ì': 'i', 'í': 'i', 'ỉ': 'i', 'ĩ': 'i', 'ị': 'i',
        'ò': 'o', 'ó': 'o', 'ỏ': 'o', 'õ': 'o', 'ọ': 'o',
        'ô': 'o', 'ồ': 'o', 'ố': 'o', 'ổ': 'o', 'ỗ': 'o', 'ộ': 'o',
        'ơ': 'o', 'ờ': 'o', 'ớ': 'o', 'ở': 'o', 'ỡ': 'o', 'ợ': 'o',
        'ù': 'u', 'ú': 'u', 'ủ': 'u', 'ũ': 'u', 'ụ': 'u',
        'ư': 'u', 'ừ': 'u', 'ứ': 'u', 'ử': 'u', 'ữ': 'u', 'ự': 'u',
        'ỳ': 'y', 'ý': 'y', 'ỷ': 'y', 'ỹ': 'y', 'ỵ': 'y'
    }
    
    for vietnamese, latin in vietnamese_chars.items():
        simplified_name = simplified_name.replace(vietnamese, latin)
    
    # Format as email
    name_parts = simplified_name.split()
    if len(name_parts) > 1:
        email_prefix = f"{name_parts[-1]}.{name_parts[0]}"
    else:
        email_prefix = name_parts[0]
    
    domains = ["gmail.com", "yahoo.com", "hotmail.com", "outlook.com", "fpt.edu.vn", "vnu.edu.vn"]
    domain = random.choice(domains)
    
    # Add random number for uniqueness
    if random.random() < 0.7:  # 70% chance to add a number
        email_prefix += str(random.randint(1, 2000))
    
    return f"{email_prefix}@{domain}"

def generate_user_info() -> Dict:
    """Generate random user information."""
    name = generate_vietnamese_name()
    gender = random.choice(["Nam", "Nữ"])
    phone = generate_vietnamese_phone()
    email = generate_email(name)
    scenario = random.choice(USER_SCENARIOS)
    
    return {
        "name": name,
        "gender": gender,
        "phone": phone,
        "email": email,
        "scenario": scenario
    }

def create_user_persona(user_info: Dict) -> str:
    """Create a detailed user persona based on basic user info."""
    system_prompt = f"""
    Create a detailed Vietnamese user persona for a mobile phone store chatbot test based on this basic information:
    - Name: {user_info['name']}
    - Gender: {user_info['gender']}
    - Phone number: {user_info['phone']}
    - Email: {user_info['email']}
    - Shopping scenario: {user_info['scenario']}
    
    Include these details in your response:
    1. Age and occupation
    2. Preferred phone brands and previous phones
    3. Budget range for a new phone
    4. Important features they're looking for (camera, battery, screen size, etc.)
    5. Communication style (formal/informal, direct/indirect, patient/impatient)
    6. How likely they are to share contact info (cautious or open)
    
    Format your response as a JSON object with these fields.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_prompt}],
            response_format={"type": "json_object"}
        )
        
        persona_json = json.loads(response.choices[0].message.content)
        return {**user_info, **persona_json}
    except Exception as e:
        logger.error(f"Error creating user persona: {e}")
        # Return basic user info if there's an error
        return user_info

def simulate_conversation(chatbot_response: str, user_persona: Dict, conversation_history: List[Dict]) -> Tuple[str, Dict]:
    """Simulate user response to the chatbot message."""
    # Extract conversation so far
    conversation_context = "\n".join([
        f"{'Chatbot' if msg['role'] == 'assistant' else 'User'}: {msg['content']}"
        for msg in conversation_history
    ])
    
    if not conversation_context:
        conversation_context = "Hội thoại mới bắt đầu."
    
    system_prompt = f"""
    Bạn đang đóng vai một khách hàng người Việt đang trò chuyện với chatbot của một cửa hàng điện thoại di động.
    
    THÔNG TIN KHÁCH HÀNG:
    {json.dumps(user_persona, ensure_ascii=False, indent=2)}
    
    LỊCH SỬ HỘI THOẠI:
    {conversation_context}
    
    TIN NHẮN MỚI NHẤT TỪ CHATBOT:
    {chatbot_response}
    
    Hãy trả lời như thể bạn là khách hàng này. Lưu ý:
    
    1. Trả lời hoàn toàn bằng tiếng Việt, phù hợp với phong cách giao tiếp của khách hàng
    2. Chỉ cung cấp số điện thoại và email nếu:
       - Chatbot đã tạo được thiện cảm
       - Bạn thực sự quan tâm đến sản phẩm được giới thiệu
       - Bạn có ý định mua hàng
    3. Phản ứng một cách tự nhiên, có thể hỏi thêm về sản phẩm, giá cả, hoặc khuyến mãi
    4. Thể hiện cá tính và mối quan tâm của khách hàng trong câu trả lời
    
    Chỉ trả về câu trả lời của khách hàng, không bao gồm giải thích nào khác.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_prompt}],
            temperature=0.7
        )
        
        user_response = response.choices[0].message.content
        
        # Extract user state for evaluation
        evaluation_prompt = f"""
        Dựa trên cuộc hội thoại sau giữa chatbot và khách hàng:
        
        {conversation_context}
        
        Chatbot: {chatbot_response}
        Khách hàng: {user_response}
        
        Hãy đánh giá trạng thái của khách hàng theo các tiêu chí sau và trả về dưới dạng JSON:
        1. buying_intention: 0-10 (mức độ khách hàng muốn mua hàng)
        2. emotional_response: "positive", "neutral", "negative"
        3. trust_level: 0-10 (mức độ tin tưởng vào chatbot)
        4. information_satisfaction: 0-10 (mức độ hài lòng với thông tin nhận được)
        5. has_provided_phone: true/false (đã cung cấp số điện thoại chưa)
        6. has_provided_email: true/false (đã cung cấp email chưa)
        7. correct_phone: true/false (nếu cung cấp, số điện thoại có đúng của khách hàng không)
        8. correct_email: true/false (nếu cung cấp, email có đúng của khách hàng không)
        """
        
        eval_response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": evaluation_prompt}],
            response_format={"type": "json_object"}
        )
        
        user_state = json.loads(eval_response.choices[0].message.content)
        
        # Validate phone numbers and emails in the response
        if user_state["has_provided_phone"]:
            # Find Vietnamese phone number patterns
            phone_match = re.search(r'0\d{9}', user_response)
            if phone_match:
                found_phone = phone_match.group(0)
                user_state["correct_phone"] = found_phone == user_persona["phone"]
        
        if user_state["has_provided_email"]:
            # Find email patterns
            email_match = re.search(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', user_response)
            if email_match:
                found_email = email_match.group(0).lower()
                user_state["correct_email"] = found_email == user_persona["email"].lower()
        
        return user_response, user_state
    
    except Exception as e:
        logger.error(f"Error simulating conversation: {e}")
        return "Xin lỗi, tôi không hiểu ý bạn. Bạn có thể nói rõ hơn được không?", {
            "buying_intention": 0,
            "emotional_response": "negative",
            "trust_level": 0,
            "information_satisfaction": 0,
            "has_provided_phone": False,
            "has_provided_email": False,
            "correct_phone": False,
            "correct_email": False
        }

def generate_initial_message(user_persona: Dict) -> str:
    """Generate the initial message from the user based on their persona."""
    system_prompt = f"""
    Tạo tin nhắn đầu tiên của khách hàng cho một cuộc trò chuyện với chatbot của cửa hàng điện thoại di động.
    
    THÔNG TIN KHÁCH HÀNG:
    {json.dumps(user_persona, ensure_ascii=False, indent=2)}
    
    Tin nhắn đầu tiên nên:
    1. Là một câu chào ngắn gọn hoặc trực tiếp vào vấn đề
    2. Thể hiện nhu cầu mua sắm của khách hàng
    3. Chưa cung cấp thông tin liên hệ (số điện thoại hoặc email)
    4. Ngắn gọn và tự nhiên, bằng tiếng Việt
    
    Chỉ trả về tin nhắn của khách hàng, không bao gồm giải thích nào khác.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_prompt}],
            temperature=0.7
        )
        
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Error generating initial message: {e}")
        return f"Chào shop, tôi đang quan tâm đến việc {user_persona['scenario'].lower()}."

def evaluate_chatbot_response(chatbot_response: str, conversation_history: List[Dict]) -> Dict:
    """Evaluate the quality of the chatbot's response."""
    # Extract conversation so far
    conversation_context = "\n".join([
        f"{'Chatbot' if msg['role'] == 'assistant' else 'User'}: {msg['content']}"
        for msg in conversation_history
    ])
    
    if not conversation_context:
        conversation_context = "Hội thoại mới bắt đầu."
    
    system_prompt = f"""
    Đánh giá chất lượng câu trả lời của chatbot dựa trên cuộc hội thoại sau:
    
    {conversation_context}
    
    Chatbot mới trả lời:
    {chatbot_response}
    
    Hãy đánh giá câu trả lời của chatbot theo các tiêu chí sau và trả về dưới dạng JSON:
    1. relevance: 0-10 (mức độ liên quan đến câu hỏi của khách hàng)
    2. helpfulness: 0-10 (mức độ hữu ích của thông tin)
    3. persuasiveness: 0-10 (khả năng thuyết phục khách hàng)
    4. language_appropriateness: 0-10 (mức độ phù hợp của ngôn ngữ)
    5. asking_for_contact: true/false (có yêu cầu thông tin liên hệ không)
    6. providing_product_info: true/false (có cung cấp thông tin sản phẩm không)
    7. emotional_appeal: 0-10 (mức độ kích thích cảm xúc tích cực)
    8. overall_score: 0-10 (điểm tổng thể)
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_prompt}],
            response_format={"type": "json_object"}
        )
        
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        logger.error(f"Error evaluating chatbot response: {e}")
        return {
            "relevance": 0,
            "helpfulness": 0,
            "persuasiveness": 0,
            "language_appropriateness": 0,
            "asking_for_contact": False,
            "providing_product_info": False,
            "emotional_appeal": 0,
            "overall_score": 0
        }

def chatbot_message(user_msg: str, conversation_history: List[Dict]) -> str:
    """
    This function should connect to your actual mobile phone store chatbot.
    For this example, we'll simulate the chatbot's response using OpenAI.
    
    In a real scenario, you would replace this with an API call to your chatbot.
    """
    # Extract conversation so far
    conversation_context = "\n".join([
        f"{'Chatbot' if msg['role'] == 'assistant' else 'User'}: {msg['content']}"
        for msg in conversation_history
    ])
    
    if not conversation_context:
        conversation_context = "Hội thoại mới bắt đầu."
        
    system_prompt = f"""
    Bạn là chatbot của một cửa hàng điện thoại di động ở Việt Nam. Nhiệm vụ của bạn là tư vấn và bán điện thoại,
    đồng thời thu thập thông tin liên hệ của khách hàng để tiếp tục tư vấn hoặc thông báo khuyến mãi.
    
    LỊCH SỬ HỘI THOẠI:
    {conversation_context}
    
    TIN NHẮN MỚI TỪ KHÁCH HÀNG:
    {user_msg}
    
    Lưu ý:
    1. Trả lời hoàn toàn bằng tiếng Việt
    2. Giới thiệu sản phẩm phù hợp với nhu cầu của khách hàng
    3. Tạo cảm xúc tích cực và xây dựng thiện cảm
    4. Cố gắng thu thập thông tin liên hệ (số điện thoại, email) của khách hàng một cách tự nhiên
    5. Nếu khách hàng đã cung cấp thông tin liên hệ, xác nhận lại thông tin đó
    
    Trả lời một cách tự nhiên, thân thiện và chuyên nghiệp.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",  # Using a different model to simulate real chatbot
            messages=[{"role": "system", "content": system_prompt}],
            temperature=0.7
        )
        
        return response.choices[0].message.content
    except Exception as e:
        logger.error(f"Error getting chatbot response: {e}")
        return "Xin lỗi quý khách, hệ thống đang gặp sự cố. Vui lòng thử lại sau."

def run_simulation(max_turns: int = 10) -> Dict:
    """Run a complete simulation and return the results."""
    # Generate user information
    user_info = generate_user_info()
    logger.info(f"Generated user info: {json.dumps(user_info, ensure_ascii=False)}")
    
    # Create detailed user persona
    user_persona = create_user_persona(user_info)
    logger.info(f"Created user persona: {json.dumps(user_persona, ensure_ascii=False)}")
    
    # Initialize conversation
    conversation_history = []
    
    # Generate initial message
    initial_message = generate_initial_message(user_persona)
    logger.info(f"Initial user message: {initial_message}")
    
    conversation_history.append({"role": "user", "content": initial_message})
    
    # Track metrics
    metrics = {
        "turns": 0,
        "collected_phone": False,
        "collected_email": False,
        "buying_intention_progression": [],
        "emotional_response_progression": [],
        "trust_level_progression": [],
        "chatbot_scores": []
    }
    
    turn = 0
    while turn < max_turns:
        turn += 1
        metrics["turns"] = turn
        
        # Get chatbot response
        chatbot_msg = chatbot_message(conversation_history[-1]["content"], conversation_history)
        logger.info(f"Turn {turn} - Chatbot: {chatbot_msg}")
        
        conversation_history.append({"role": "assistant", "content": chatbot_msg})
        
        # Evaluate chatbot response
        chatbot_evaluation = evaluate_chatbot_response(chatbot_msg, conversation_history[:-1])
        metrics["chatbot_scores"].append(chatbot_evaluation)
        logger.info(f"Chatbot evaluation: {json.dumps(chatbot_evaluation, ensure_ascii=False)}")
        
        # Get user response
        user_msg, user_state = simulate_conversation(chatbot_msg, user_persona, conversation_history)
        logger.info(f"Turn {turn} - User: {user_msg}")
        logger.info(f"User state: {json.dumps(user_state, ensure_ascii=False)}")
        
        # Update metrics
        metrics["buying_intention_progression"].append(user_state["buying_intention"])
        metrics["emotional_response_progression"].append(user_state["emotional_response"])
        metrics["trust_level_progression"].append(user_state["trust_level"])
        
        if user_state["has_provided_phone"] and user_state["correct_phone"]:
            metrics["collected_phone"] = True
        
        if user_state["has_provided_email"] and user_state["correct_email"]:
            metrics["collected_email"] = True
        
        # Add user message to history
        conversation_history.append({"role": "user", "content": user_msg})
        
        # Check if we've collected all needed information
        if metrics["collected_phone"] and metrics["collected_email"]:
            logger.info("Successfully collected both phone and email. Ending simulation.")
            break
        
        # Check if user is disengaging
        if user_state["buying_intention"] < 3 and turn > 3:
            logger.info("User has low buying intention after multiple turns. Ending simulation.")
            break
    
    # Final summary
    metrics["success"] = metrics["collected_phone"] and metrics["collected_email"]
    metrics["conversation"] = conversation_history
    
    return metrics

def run_multiple_simulations(num_simulations: int = 10) -> Dict:
    """Run multiple simulations and aggregate results."""
    all_results = []
    successful_simulations = 0
    total_turns = 0
    phone_collection_rate = 0
    email_collection_rate = 0
    
    for i in range(num_simulations):
        logger.info(f"Starting simulation {i+1}/{num_simulations}")
        result = run_simulation()
        all_results.append(result)
        
        if result["success"]:
            successful_simulations += 1
        
        total_turns += result["turns"]
        
        if result["collected_phone"]:
            phone_collection_rate += 1
        
        if result["collected_email"]:
            email_collection_rate += 1
    
    # Calculate aggregate metrics
    aggregate_results = {
        "total_simulations": num_simulations,
        "success_rate": successful_simulations / num_simulations,
        "average_turns": total_turns / num_simulations,
        "phone_collection_rate": phone_collection_rate / num_simulations,
        "email_collection_rate": email_collection_rate / num_simulations,
        "detailed_results": all_results
    }
    
    # Save results to file
    with open(f"chatbot_evaluation_results_{int(time.time())}.json", "w", encoding="utf-8") as f:
        json.dump(aggregate_results, ensure_ascii=False, indent=2, fp=f)
    
    return aggregate_results

def analyze_results(results: Dict) -> Dict:
    """Analyze the simulation results and provide insights."""
    system_prompt = f"""
    Analyze the following chatbot evaluation results and provide insights:
    
    {json.dumps(results, ensure_ascii=False, indent=2)}
    
    Focus on:
    1. Overall performance analysis
    2. Key strengths of the chatbot
    3. Areas for improvement
    4. Specific recommendations to increase contact information collection
    5. Patterns in user emotional responses and buying intentions
    
    Format your response as a JSON object with these sections.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[{"role": "system", "content": system_prompt}],
            response_format={"type": "json_object"}
        )
        
        analysis = json.loads(response.choices[0].message.content)
        
        # Save analysis to file
        with open(f"chatbot_analysis_{int(time.time())}.json", "w", encoding="utf-8") as f:
            json.dump(analysis, ensure_ascii=False, indent=2, fp=f)
        
        return analysis
    
    except Exception as e:
        logger.error(f"Error analyzing results: {e}")
        return {
            "error": str(e),
            "analysis_status": "failed"
        }

def main():
    """Main function to run the chatbot evaluation."""
    logger.info("Starting Mobile Phone Store Chatbot Evaluation")
    
    # Set number of simulations
    num_simulations = 10
    
    # Run simulations
    results = run_multiple_simulations(num_simulations)
    logger.info(f"Completed {num_simulations} simulations with success rate: {results['success_rate']}")
    
    # Analyze results
    analysis = analyze_results(results)
    logger.info("Analysis complete")
    
    # Print summary
    print("\n===== CHATBOT EVALUATION SUMMARY =====")
    print(f"Total simulations: {results['total_simulations']}")
    print(f"Success rate: {results['success_rate'] * 100:.2f}%")
    print(f"Average turns per conversation: {results['average_turns']:.2f}")
    print(f"Phone collection rate: {results['phone_collection_rate'] * 100:.2f}%")
    print(f"Email collection rate: {results['email_collection_rate'] * 100:.2f}%")
    print("\nDetailed analysis saved to chatbot_analysis_[timestamp].json")
    print("Raw results saved to chatbot_evaluation_results_[timestamp].json")

main()

2025-05-13 22:02:21,668 - __main__ - INFO - Starting Mobile Phone Store Chatbot Evaluation
2025-05-13 22:02:21,670 - __main__ - INFO - Starting simulation 1/10
2025-05-13 22:02:21,673 - __main__ - INFO - Generated user info: {"name": "Hoàng Trang", "gender": "Nam", "phone": "0335526884", "email": "trang.hoang1608@yahoo.com", "scenario": "Đang tìm một chiếc điện thoại mới với camera tốt"}
2025-05-13 22:02:29,817 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-05-13 22:02:29,832 - __main__ - INFO - Created user persona: {"name": "Hoàng Trang", "gender": "Nam", "phone": "0335526884", "email": "trang.hoang1608@yahoo.com", "scenario": "Đang tìm một chiếc điện thoại mới với camera tốt", "userPersona": {"name": "Hoàng Trang", "gender": "Nam", "phoneNumber": "0335526884", "email": "trang.hoang1608@yahoo.com", "age": 28, "occupation": "Nhân viên marketing", "preferredPhoneBrands": ["Samsung", "Apple", "Xiaomi"], "previousPhones": ["Samsung G

KeyboardInterrupt: 